In [1]:
import cv2
import numpy as np

In [2]:
def ssd_naive(strip, template):
    template_cols = template.shape[1]
    strip_cols = strip.shape[1]
    s = np.zeros(strip_cols - template_cols + 1)
    for i in range(strip_cols - template_cols + 1):
        s[i] = np.sum((template - strip[:, i:i+template_cols]) ** 2)
    return s[None,:]
                      

def disparity_ssd(L, R, template_size=5, disparity_range=100):
    """Compute disparity map D(y, x) such that: L(y, x) = R(y, x + D(y, x))
    
    Params:
    L: Grayscale left image
    R: Grayscale right image, same size as L

    Returns: Disparity map, same size as L, R
    """
    
    image_rows, image_cols = L.shape
    D = np.zeros(L.shape)
    
    for row in np.arange(image_rows - template_size + 1):     
        for col in np.arange(image_cols - template_size + 1):
            row_max = row + template_size
            col_max = col + template_size
            template = L[row:row_max, col:col_max].astype(np.float32)
            col_mid = (col + col_max) / 2
            strip_col_min = int(max(col_mid - disparity_range/2, 0))
            strip_col_max = int(min(col_mid + disparity_range/2, image_cols))
            R_strip = R[row:row_max, strip_col_min:strip_col_max].astype(np.float32)
            # cost = ssd_naive(R_strip, template) # slow
            cost = cv2.matchTemplate(R_strip, template, method=cv2.TM_SQDIFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(cost)
            R_match_col = strip_col_min + min_loc[0]
            D[row, col] = R_match_col - col
            
    return D